In [1]:
dia = ''

In [2]:
# Parameters
dia = "2024-05-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Jogos do dia 2024-05-23



The msedgedriver version (124.0.2476.0) detected in PATH at C:\Users\johnn\OneDrive\Documentos\apostas\basketball_models\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (125.0.2535.51); currently, msedgedriver 125.0.2535.51 is recommended for MicrosoftEdge 125.*, so it is advised to delete the driver in PATH and retry


  0%|                                                                                           | 0/47 [00:00<?, ?it/s]

  2%|█▊                                                                                 | 1/47 [00:00<00:27,  1.67it/s]

  4%|███▌                                                                               | 2/47 [00:01<00:35,  1.28it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 46/47 [00:39<00:00,  1.14it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:40<00:00,  1.01s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:40<00:00,  1.15it/s]

Dataset de temporada atualizado com 41 jogos.

----------------------------------------------------------

Atenção! 6 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
11,2024-05-23,Áustria Superliga,13:00,UBSC Graz,Oberwart,NaN,NaN,Não iniciado
27,2024-05-23,Brasil Cbb,19:30,Osasco,Tatuí,NaN,NaN,Não iniciado
33,2024-05-23,Argentina Liga A,20:00,Ferro,San Lorenzo,NaN,NaN,Não iniciado
35,2024-05-23,Colômbia Lbp,20:15,Caribbean Storm Islands,Corsarios Cartagena,NaN,NaN,Não iniciado
40,2024-05-23,Argentina Liga A,21:00,Comunicaciones Mercedes,Boca Juniors,NaN,NaN,Não iniciado
42,2024-05-23,Argentina Liga A,22:10,Quimsa,Olimpico,NaN,NaN,Não iniciado
